In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "../src"))

In [2]:
import logging
import jax
import jax.numpy as np
import numpy as onp
import skbio.stats.composition as cmp
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import statsmodels.api as sm
from datetime import datetime

from simulate_data_fct import sim_IV_ilr_linear
from plot_fct import update_layout, img_path, update_layout_px, colours, plot_ilr_X_vs_Y
from run_methods_all import run_methods_confidence_interval
from plot_fct import plot_mse_results, plot_beta_results

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# set up logging file
logging.getLogger().setLevel(logging.INFO)

In [6]:
key = jax.random.PRNGKey(191)

# Linear Setup

## Data Generation

In [12]:
n = 1000
p = 30
num_inst = 20
instrument_strength = "unknown"
mypath = os.path.join(img_path, "HighDimLinear")


V = cmp._gram_schmidt_basis(p)

c_X = np.hstack([np.array([-1, 2, -2, 1]), np.zeros(p-1-4, )]) # confounder multiplication to X

alpha0 = np.hstack([np.array([1, 1, 3, 1]), np.zeros(p-1-4, )])
alphaT = jax.random.choice(key, np.array([0.5, 0.75, 0.25, 0, 0, 0]), (p-1, num_inst))  # -4, 4

betaT_log = np.hstack([np.array([5, -5, 3, -3]), np.zeros(p-1-4, )])
betaT_p = np.hstack([betaT_log, -betaT_log.sum()])
beta0 = 5
betaT = V@betaT_p

mu_c = 3
c_Y = 4

In [8]:
confounder, Z_sim, X_sim, Y_sim, X_star, Y_star = sim_IV_ilr_linear(
            key,
            n=n,
            p=p,
            num_inst=num_inst,
            mu_c=mu_c,
            c_X=c_X,
            alpha0=alpha0,
            alphaT=alphaT,
            c_Y=c_Y,
            beta0=beta0,
            betaT=betaT,
            num_star=500)
X_sim_ilr = cmp.ilr(X_sim)
X_star_ilr = cmp.ilr(X_star)

## Data Visualization

In [46]:
fig = px.bar(pd.DataFrame(X_sim[:100, :], columns=["Microbiota "+str(i) for i in range(X_sim.shape[1])]))
fig = update_layout_px(fig)
fig.update_xaxes(title="Sample number")
fig.update_yaxes(title="Percentage of microbiota in sample")
fig.write_image(os.path.join(mypath,
                             "Barplot_Sample_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

In [10]:
# Compute F-Statistics 
ZZ_sim = onp.array(sm.add_constant(Z_sim))
alphahat = np.linalg.inv(ZZ_sim.T@ZZ_sim)@ZZ_sim.T@X_sim_ilr
MSM = np.sum(((ZZ_sim@alphahat - np.mean(X_sim_ilr, axis=0))**2), axis=0)/(num_inst)
MSE = np.sum(((ZZ_sim@alphahat - X_sim_ilr)**2), axis=0)/(n-num_inst)
F = MSM / MSE
F

DeviceArray([1.0789142e+01, 3.0695031e+00, 4.8881588e+00, 1.1596719e+01,
             1.8656028e+11, 1.7220629e+11, 2.3095638e+11, 1.6802064e+11,
             1.5984609e+11, 1.5448152e+11, 1.0957689e+11, 2.5182944e+11,
             3.7260768e+11, 2.3788755e+11, 2.6528165e+11, 1.5829606e+11,
             2.2666317e+11, 1.8636797e+11, 1.1781357e+11, 2.0500675e+11,
             3.2960496e+11, 1.5847331e+11, 1.2177576e+11, 1.2134960e+11,
             1.3080679e+11, 2.1113422e+11, 2.1239371e+11, 1.4785556e+11,
             2.6978796e+11], dtype=float32)

In [24]:
fig = plot_ilr_X_vs_Y(X_sim_ilr, Y_sim, Y_star)

fig.update_layout(xaxis3=dict(showline=True, linewidth=2, linecolor="black"))
fig.update_layout(yaxis3=dict(showline=True, linewidth=2, linecolor="black"))
fig.update_layout(xaxis4=dict(showline=True, linewidth=2, linecolor="black"))
fig.update_layout(yaxis4=dict(showline=True, linewidth=2, linecolor="black"))
fig.update_layout(xaxis5=dict(showline=True, linewidth=2, linecolor="black"))
fig.update_layout(yaxis5=dict(showline=True, linewidth=2, linecolor="black"))

fig.write_image(os.path.join(mypath,
                             "Data_vs_trueeffect_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))


fig.show()

# Diversity Approach

In [25]:
from plot_fct import plot_diversity_methods
from helper_fct import diversity
from run_methods_all import run_diversity_estimation_methods
div_shannon = diversity(X_sim, "shannon")
div_simpson = diversity(X_sim, "simpson")

In [26]:
# Shannon Estimation
x, y, ytrue, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results = run_diversity_estimation_methods(
    Z_sim, div_shannon, Y_sim, Ytrue=Y_star, methods=["OLS", "2SLS", "KIV"])

fig = plot_diversity_methods(x, y, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results, ytrue)
fig.write_image(os.path.join(mypath,
                             "ShannonDiversity_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

INFO:absl:Setup matrices...


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     4.733
Date:                 Di, 01 Jun 2021   Prob (F-statistic):           4.61e-11
Time:                        15:43:53   Log-Likelihood:                -1372.8
No. Observations:                1000   AIC:                             2788.
Df Residuals:                     979   BIC:                             2891.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2798      0.250      5.110      0.0

INFO:absl:Optimize lambda...
INFO:absl:Optimal lambda [4.52595122]...
INFO:absl:Optimize xi...
INFO:absl:Optimal xi [-5.21784051]...
INFO:absl:Predict treatment effect...


In [27]:
# Simpson Estimation
x, y, ytrue, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results = run_diversity_estimation_methods(
    Z_sim, div_simpson, Y_sim, Ytrue=Y_star, methods=["OLS", "2SLS", "KIV"])

plot_diversity_methods(x, y, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results, ytrue)
fig.write_image(os.path.join(mypath,
                             "SimpsonDiversity_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

INFO:absl:Setup matrices...


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     2.155
Date:                 Di, 01 Jun 2021   Prob (F-statistic):            0.00234
Time:                        15:43:56   Log-Likelihood:                -1397.4
No. Observations:                1000   AIC:                             2837.
Df Residuals:                     979   BIC:                             2940.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3727      0.257      1.452      0.1

INFO:absl:Optimize lambda...
INFO:absl:Optimal lambda [4.96727534]...
INFO:absl:Optimize xi...
INFO:absl:Optimal xi [-4.17793788]...
INFO:absl:Predict treatment effect...


# Confidence Intervals for higher dimensional methods

In [28]:
from run_methods_all import run_methods_all

In [29]:
today = datetime.now()
num_iteration = 20

path = os.getcwd()
mypath = os.path.join(path, "temp",
                      today.strftime("%Y_%m_%d_%H_%M") + "_LINEAR_ConfidenceInterval_"+str(instrument_strength)
                      +"_"+ str(num_iteration)+"_Microbiota_" +str(p))

if not os.path.isdir(mypath):
    os.makedirs(mypath)


# create logging file with relevant information
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M',
                    filename=os.path.join(os.getcwd(), mypath, "configurations.log"),
                    filemode='w'
                    )

logging.info("n="+str(n)+", p="+str(p)+", instrument strength: "+instrument_strength)
logging.info("Number of runs: "+str(num_iteration))
logging.info("First Stage Specification: ")
logging.info("Number of Instruments: "+str(num_inst))
logging.info("Instrument Strength by F-Test: " +str(F))
logging.info("alpha0="+str(alpha0)+", alphaT="+str(alphaT)+", confounder multiplicator c_X="+str(c_X))
logging.info("Second Stage Specification: ")
logging.info("beta0="+str(beta0)+", betaT="+str(betaT)+", confounder multiplicator c_Y="+str(c_Y))
logging.info("Confounder mean mu_c="+str(mu_c))


df_beta, df_mse, mse_large_confidence= run_methods_confidence_interval(
    key, num_iteration, 
    n, p, num_inst, mu_c, c_X, alpha0, alphaT, c_Y, beta0, betaT, num_star=200)

**************************************************************************************************
*****************************************We are at 0 of 20***********************************************
**************************************************************************************************
---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [ 7.071068  -2.4494898  3.4641016  0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.       ]
Estimated Beta: [ 6.34 -1.48  2.89  0.18 -0.46  0.67  0.65 -0.97  1.16  0.31 -0.61  0.43
 -0.68 -0.56  0.7   0.   -0.26  0.01  0.63  1.48  1.72 -0.82 -0.03 -1.04
  0.06  0.14 -0.2  -0.51 -1.37]
Estim

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/base/model.py:1832: ValueWarning:

covariance of constraints does not have full rank. The number of constraints is 29, but rank is 9

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1787: RuntimeWarning:

invalid value encountered in sqrt



                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                      -0.047
Model:                         IV2SLS   Adj. R-squared:                 -0.078
Method:                     Two Stage   F-statistic:                -0.0009441
                        Least Squares   Prob (F-statistic):               1.00
Date:                 Di, 01 Jun 2021                                         
Time:                        15:44:26                                         
No. Observations:                1000                                         
Df Residuals:                     970                                         
Df Model:                          29                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         56.6557   9.42e+07   6.02e-07      1.0

Error: 331.38

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY SECOND STAGE - Kernel Regression KIV >>>>>>>>>>>>>>>>>>>>>>>>>
Error: 16.36

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Kernel Regression KIV (manual) >>>>>>>>>>>>>>>>>>>>>>>>>
Error: 229.82

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - ILR ILR Regression Implementation >>>>>>>>>>>>>>>>>>>>>>>


/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/base/model.py:1832: ValueWarning:

covariance of constraints does not have full rank. The number of constraints is 29, but rank is 9



...

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/base/model.py:1832: ValueWarning:

covariance of constraints does not have full rank. The number of constraints is 29, but rank is 13

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1787: RuntimeWarning:

invalid value encountered in sqrt



                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.825
Model:                         IV2SLS   Adj. R-squared:                  0.819
Method:                     Two Stage   F-statistic:                   -0.3683
                        Least Squares   Prob (F-statistic):               1.00
Date:                 Di, 01 Jun 2021                                         
Time:                        15:50:55                                         
No. Observations:                1000                                         
Df Residuals:                     970                                         
Df Model:                          29                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.3776        nan        nan        n

Error: 494.65

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY SECOND STAGE - Kernel Regression KIV >>>>>>>>>>>>>>>>>>>>>>>>>
Error: 18.76

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Kernel Regression KIV (manual) >>>>>>>>>>>>>>>>>>>>>>>>>
Error: 332.41

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - ILR ILR Regression Implementation >>>>>>>>>>>>>>>>>>>>>>>


/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/base/model.py:1832: ValueWarning:

covariance of constraints does not have full rank. The number of constraints is 29, but rank is 20

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1787: RuntimeWarning:

invalid value encountered in sqrt



                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.998
Model:                         IV2SLS   Adj. R-squared:                  0.998
Method:                     Two Stage   F-statistic:                     338.7
                        Least Squares   Prob (F-statistic):               0.00
Date:                 Di, 01 Jun 2021                                         
Time:                        15:51:15                                         
No. Observations:                1000                                         
Df Residuals:                     970                                         
Df Model:                          29                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.9880   7.49e+05   5.32e-06      1.0

In [30]:
import pickle
results = {
    "df_mse": df_mse,
    "df_beta": df_beta,
    "mse_large": mse_large_confidence
}

with open(os.path.join(mypath, "results_"+str(p)+str(instrument_strength)+".pickle"), "wb") as f:
    pickle.dump(results, f)
    f.close()


# Mean Squarred Error Plot

In [36]:
filter_list = ["ALR+LC", "ILR+ILR", "ONLY Second ILR", "ONLY Second LC", "DIR+LC", "ILR+LC"]

fig = plot_mse_results(df_mse, filter_list)
fig.update_yaxes(range=(0, 50))
fig.write_image(os.path.join(mypath,
                             "MSEFull_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+"_3.pdf" ))
fig.show()

# Beta Plot

In [39]:
fig = plot_beta_results(df_beta, V.T@betaT, filter_list)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_yaxes(range=(-15, 20))
fig.write_image(os.path.join(mypath,
                             "BetaFull_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()


In [43]:
fig = plot_beta_results(df_beta, V.T@betaT, filter_list, beta_zero=True)


fig.update_yaxes(range=(-5, 5))
fig.write_image(os.path.join(mypath,
                             "BetaZeroFull_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()
